In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

In [2]:
df_relationships = pd.read_excel('data/person_relationships.xlsx')
df_contracts = pd.read_excel('data/contracts.xlsx')
df_person_mentions = pd.read_excel('data/person_mentions.xlsx')

In [ ]:
all_person_ids = df_relationships['Person 1 ID'].tolist() + df_relationships['Person 2 ID'].tolist() + df_relationships['Person 1 Name'].tolist() 
print(len(all_person_ids)- len(set(all_person_ids)))
print(len(all_person_ids))





20846
238872


In [15]:
is_father_network = df_relationships[df_relationships['Relationship Type'] == 'grz:isFatherOf']
is_father_network

,Person 1 ID,Person 1 Name,Relationship Type,Person 2 ID,Person 2 Name
1,d9838b83-e2c7-42c8-b036-6799d5ca734a,Abbondio D'Apiano,grz:isFatherOf,906b7b06-0ed8-4c9e-b525-88f48d5ca056,Francesco D'Apiano
3,8bf2a7c7-4797-439e-8ed5-6717a4eaf51d,Abbondio Gracio,grz:isFatherOf,8d55328a-e415-4d42-a9ca-e048a3c12970,Bernardo Gracio
13,ba63b706-3020-46d4-9bd7-8b813029b9b4,Abramo 162446,grz:isFatherOf,99b83179-b185-421a-a1fd-75a16cbc8528,Cristoforo 162446
21,32a926ed-ec94-4575-abbf-c56ec67a3ebb,Achille 153-II306,grz:isFatherOf,2773b53e-7472-4d8d-8436-f05b747e6b9f,Pomponio di Achille (marangon)
24,e01d299f-b702-46b9-bf67-74b6ba2ea873,Adamo 1,grz:isFatherOf,45026ded-8e12-4f36-9d81-984f6acdae49,Andrea di Adamo (tellariol)
...,...,...,...,...,...
79603,cd637d1a-18a7-41b1-8c33-fdb41ad21ee3,Zubiet?,grz:isFatherOf,76e0be63-31ef-4382-9565-ec06a8edfd06,Silvestro di Zubiet?
79604,f8fd0f70-5432-43fa-af39-82464360cf17,Zubit?,grz:isFatherOf,76e0be63-31ef-4382-9565-ec06a8edfd06,Silvestro di Zubiet?
79607,a0a5afcb-39ce-420d-8257-3a7f4b44e512,Zulian delli Rusi,grz:isFatherOf,b0553264-22f7-4089-a986-a951b86a9271,Gasparo delli Rusi
79611,24f4759f-7fe4-4c92-afbc-52e2bbac919f,Zulian Furlan,grz:isFatherOf,713fd39d-dc0e-4c93-8086-d58b66f939ff,Lorenzo Furlan (tiraoro)


In [25]:
df_masters_apprantices = df_person_mentions[df_person_mentions['Tag'].isin(['grz:Master', 'grz:Apprentice'])][colum_select]
df_masters_apprantices

,Person ID,Person Name,Contract ID,Tag
0,f1e0fa88-1ff5-4cc7-8fba-854d148f0306,Paolo Di Bori,f47a0743-e315-4606-bfab-8f603d5000f9,grz:Apprentice
2,bc3ecc02-652f-42d9-8bba-fc4561152b5a,Nicolò Barazzuso,d74fb5f0-ab73-4e2e-9762-3227294d6899,grz:Master
4,49f8b298-b445-4715-8feb-97b23ce2770d,Giacomo Durin,197bdfc9-c26a-417f-b0ba-657b81a16cac,grz:Apprentice
5,065fb6cf-0990-4acb-bf19-bf6b97471767,Battista Crovato 179149,63bb2629-2e05-4df6-a619-fcb42df842a0,grz:Apprentice
6,bc2fa44a-b7db-4aa7-b39f-c02ba5b43220,Gaspare Homo,d78a05f4-2a87-4cea-928d-460418f5e65b,grz:Apprentice
...,...,...,...,...
156140,ac44d888-933b-412d-9d1e-22c8b87110cb,Agostino de Francesco,1fe25c35-f56d-4f13-afe2-1cf3f05226fc,grz:Master
156141,f6a5ae1a-c5ba-4d3d-b7d3-b3a59f61d48a,Giovanni Antonio Marchese,263a1e09-f4c0-4214-8b09-d252256f26bb,grz:Apprentice
156142,2d16203d-1180-4513-96d7-f76b2f4cec3b,Giovanni Battista Panciera (sartor),870c156d-dea4-4b64-b44e-9c54867a5c4d,grz:Apprentice
156145,23b1a1f8-1602-470b-b98d-1ccf67e8ce88,Giuseppe Marzini,a88b5738-27ea-41a9-87cb-0c9026eb040b,grz:Apprentice


In [23]:
colum_select = ['Person ID', 'Person Name', 'Contract ID', 'Tag']
df_masters = df_person_mentions[df_person_mentions['Tag'].isin(['grz:Master'])][colum_select]
df_apprantices = df_person_mentions[df_person_mentions['Tag'].isin(['grz:Apprentice'])][colum_select]

df_master_apprentice = pd.merge(
    df_masters,
    df_apprantices,
    on='Contract ID',
    suffixes=('_master', '_apprentice')
)
df_master_apprentice

,Person ID_master,Person Name_master,Contract ID,Tag_master,Person ID_apprentice,Person Name_apprentice,Tag_apprentice
0,bc3ecc02-652f-42d9-8bba-fc4561152b5a,Nicolò Barazzuso,d74fb5f0-ab73-4e2e-9762-3227294d6899,grz:Master,9073a473-e3fa-49c3-ac7a-2d5d48523e61,Pietro di Domenico 174097,grz:Apprentice
1,0e752d6a-c511-4059-94d8-efe086c7b4a9,Ancillo (frutariol),183f015d-a487-489a-b11c-9406609d77a4,grz:Master,0bd14817-aae6-44fb-8a1a-72889ccc9871,Agostino Di Bechi,grz:Apprentice
2,c89a2941-5d6d-4f56-a617-518f31e2bea3,Domenico di Antonio (sartor),ab217f68-1ec7-4017-b70e-3b5adcb03756,grz:Master,74427903-a6e5-44bd-a5da-1fd4f1ecdf14,Pietro di Domenico 151270,grz:Apprentice
3,653adad8-9b90-4fbd-97ab-3e14686553b2,Paolo Algarotti,b3a7ee3f-d9f2-40c2-b12e-e2f6735972f5,grz:Master,94131b97-9bc7-4735-a9be-b6dad07de56e,Antonio Piceni 172190,grz:Apprentice
4,bf4098fb-20a4-4b8a-a5f4-52e8abc76989,Giuseppe Spin,3b3e0754-b286-4885-8bd0-1f0654b078b1,grz:Master,94795140-83c5-4902-bd5d-9b1144c51388,Giuseppe Di Gariri?,grz:Apprentice
...,...,...,...,...,...,...,...
54785,a0a0071d-ad71-485b-991d-7f9922bbd672,Piero Menardi,e961f731-5bfd-47e4-861f-05b0c0904cf1,grz:Master,ba4c857c-985f-4eeb-b8db-d1bc62432fea,Zuane Azalle,grz:Apprentice
54786,d2bfdaeb-5601-4be3-b0e9-d473aad0fda0,Paolo Capenir,81db0496-0d79-467c-8586-7061e5a16453,grz:Master,8e9bc84a-f2d3-4afc-8e90-931b70b2ec23,Giacomo Di Barlieri,grz:Apprentice
54787,b412c9c5-f582-4916-93c4-180e34398a9c,Gerolamo Boldrini,40ad9c6e-d121-4d7e-8a0c-0e0601e05612,grz:Master,8029e1ba-ca84-4ab7-a8e7-089d51ee4520,Nicolò Taccolin,grz:Apprentice
54788,a73f3d96-1120-4066-bd20-4fb636bc29c7,Domenico di Paolo (boter),a739f105-9c71-4ceb-b369-3de4c271a83a,grz:Master,1fab2829-c6a1-4588-bc85-251bc8d18735,Pietro del Mondanese,grz:Apprentice
